In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd

# Para visualización de datos
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

# Para cargar los datos
# -----------------------------------------------------------------------
import pickle
import sys
import os
from time import sleep
sys.path.append(os.path.abspath('../'))
# Para sistemas de recomendacion basados en cotenido
# -----------------------------------------------------------------------
from scipy.spatial.distance import pdist , squareform   # Para calcular la distancia entre puntos 

# Importamos nuestras funciones
# -----------------------------------------------------------------------
from src import soporte_sistemas_recomendacion as sr
from src import soporte_tablas as st

# Configuración de pandas
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)

In [2]:
conexion = st.conexion("IMDB")
cursor = conexion.cursor()  

In [3]:
cursor.execute('SELECT * FROM "pelicula" pe, "artistas_detalles" ad  where pe."Id"= ad."Id" ;')  
datos_ejercicios = cursor.fetchall() 
columnas_ejercicios = [desc[0] for desc in cursor.description]  
df_notas = pd.DataFrame(datos_ejercicios, columns=columnas_ejercicios) 


In [4]:
metadata_final = df_notas[['Tipo','Titulo','generos','cali_imdb']]

In [5]:
# seleccionamos las columnas de interés y renombramos el nombre de las columnas
df_collab = metadata_final[['Tipo','Titulo','generos','cali_imdb']]

df_collab.head()

,Tipo,Titulo,generos,cali_imdb
0,Movie,Las Travesuras de Super Chido,Action,7.7
1,Movie,Velai Kidaichiruchu,Action,5.2
2,Movie,Sor Batalla,Action,6.6
3,Movie,Azaad Desh Ke Gulam,Action,5.3
4,Movie,Suo ming fei dao,Action,NaN


In [21]:
# pivitamos la tabla
userRatings = df_collab.pivot_table(index=['Titulo'],columns=['generos'],values='cali_imdb').fillna(0)
userRatings.head()


generos,Action,Comedy,Drama,Fantasy,Horror,Mystery,Romance,Thriller
Titulo,,,,,,,,
'Ne günstige Gelegenheit,0.0,4.9,0.0,0.0,0.0,0.0,0.0,0.0
12 Feet Deep,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.2
12/12/2012,0.0,0.0,0.0,0.0,1.9,0.0,0.0,0.0
13 segundos,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.2
14 Days in Paradise,0.0,0.0,0.0,0.0,0.0,0.0,4.7,0.0


In [28]:
# calculamos la matriz de correlacion
matr_corre = userRatings.corr(method='pearson')
matr_corre.head()

generos,Action,Comedy,Drama,Fantasy,Horror,Mystery,Romance,Thriller
generos,,,,,,,,
Action,1.000000,-0.113113,-0.116912,-0.045063,-0.128707,-0.085678,-0.078458,-0.120026
Comedy,-0.113113,1.000000,-0.187795,-0.072385,-0.206741,-0.137625,-0.126026,-0.192796
Drama,-0.116912,-0.187795,1.000000,-0.074816,-0.213684,-0.142247,-0.130258,-0.199271
Fantasy,-0.045063,-0.072385,-0.074816,1.000000,-0.082364,-0.054829,-0.050208,-0.076808
Horror,-0.128707,-0.206741,-0.213684,-0.082364,1.000000,-0.156597,-0.143399,-0.219374


In [29]:
nombre_movie = 'Action'
rating = 5

similar_ratings = matr_corre[nombre_movie]*(rating-2.5)
recommendacion = similar_ratings.sort_values(ascending=False)

# printemoas la recomendacion
print(recommendacion[1:11].sort_values(ascending=False))

generos
Fantasy    -0.112659
Romance    -0.196144
Mystery    -0.214196
Comedy     -0.282783
Drama      -0.292280
Thriller   -0.300064
Horror     -0.321766
Name: Action, dtype: float64
